<img src="https://static.wixstatic.com/media/f11c7c_69283cb47c30496d953e1d8d4c6b5b18~mv2.png/v1/fill/w_75,h_75,al_c,usm_0.66_1.00_0.01/Logo%20new.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />
     
# Segmenting and clustering Toronto's neighborhoods

This notebook holds my final assignment for the IBM Data Science certificate.

---

## Introduction to this notebook
This notebook holds my code for the Peer-graded assignment for:<br>
*Applied Data Science capstone | part of the IBM Data Science Professional Certifcate course | week 3*

## My code

### setup of environment

In [121]:
# import modules
import pandas as pd

# ensure the SSL error from the read_html is fixed
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### get the data from the URL into a dataframe

In [122]:
# read the data from the provided location and use the first table on the Wiki page
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df.head())
print(df.shape)
print()

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
(180, 3)



### pre-process check and the data

In [123]:
#check nr of columns
col = 3
log_entry = 'Number of columns equals:'
if len(df.columns) == col:
    print('{} {} | correct'.format(log_entry,col))
else:
    print('{} {} | something is wrong, please check'.format(log_entry,col))
print()

Number of columns equals: 3 | correct



In [124]:
# skip rows with Not Assigned for Borough
row = df.loc[df.Borough == 'Not assigned', 'Borough'].count()
log_entry = "Number of Borough rows with 'Not assigned' equals:"
if row == 0:
    print('{} {} | correct'.format(log_entry,row))
else:
    print('{} {} | please pre-process the rows'.format(log_entry,row))
print()

df = df[df['Borough']!='Not assigned']
print(df.head())
print('Shape after skipping rows: {}'.format(df.shape))
print()

# check after skipping
row = df.loc[df.Borough == 'Not assigned', 'Borough'].count()
log_entry = "Number of Borough rows with 'Not assigned' equals:"
if row == 0:
    print('{} {} | correct'.format(log_entry,row))
else:
    print('{} {} | please pre-process the rows'.format(log_entry,row))
print()

Number of Borough rows with 'Not assigned' equals: 77 | please pre-process the rows

  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
Shape after skipping rows: (103, 3)

Number of Borough rows with 'Not assigned' equals: 0 | correct



In [125]:
# find the duplicates on neighborhood entries and return all rows
# duplicates on neighbourhood means the neighbourhood covers more than one postal code
# just a check, no further processing needed

key = 'Neighbourhood'
df_dups = df[df.duplicated([key], keep=False)]
if len(df_dups) > 0:
    print(df_dups)
    print('Duplicates on {} found | no action needed, correct'.format(key))
else:
    print('No duplicates on {} found | correct'.format(key))
print()

# find the duplicates on postal code entries (ie neighbourhoods that should be combined)
key = 'Postal Code'
df_dups = df[df.duplicated([key], keep=False)]
if len(df_dups) > 0:
    print(df_dups)
    # iterate through the duplicates Postal Codes
    for index, row in df_dups.iterrows():
        to_cat = df.loc[df['Postal Code'] == row['Postal Code'], 'Neighbourhood']
        # concate their values
        to_cat = to_cat.str.cat(sep=', ')
        # update DataFrame
        df.loc[df['Postal Code'] == row['Postal Code'], 'Neighbourhood'] = to_cat.str.cat(sep=', ')
    # all duplicates are aligned to they all can be removed but the first one
    df.drop_duplicates(subset=['Postal Code'], inplace=True)
    print('Duplicates on {} found and fixed | correct'.format(key))
    print(df.shape)
else:
    print('No duplicates on {} found | correct'.format(key))

print()

# find the postal codes with more than one neighbourhood (ie with a comma in the neigbourhood)
df_dups = df[df['Neighbourhood'].str.contains(',')]
if len(df_dups) > 0:
    print(df_dups)
    print('{} postal codes with multiple neighbourhoods found | no action needed, correct'.format(len(df_dups)))
else:
    print('No entries with ,')
print()

   Postal Code     Borough Neighbourhood
11         M3B  North York     Don Mills
20         M3C  North York     Don Mills
65         M3K  North York     Downsview
74         M3L  North York     Downsview
83         M3M  North York     Downsview
92         M3N  North York     Downsview
Duplicates on Neighbourhood found | no action needed, correct

No duplicates on Postal Code found | correct

    Postal Code           Borough  \
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
8           M9A         Etobicoke   
9           M1B       Scarborough   
..          ...               ...   
157         M5X  Downtown Toronto   
160         M8X         Etobicoke   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                         Neighbourhood  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrenc

In [126]:
# if a cell has a Borough with a 'Not assigned' neighborhood, 
# the neighborhood must become the same as the borough

df_dups = df[df['Neighbourhood'] == 'Not assigned']
if len(df_dups) > 0:
    print(df_dups)
    #iterate through the rows
    for index, row in df_dups.iterrows():
        #change the Neighbourhood value into the Borough value where applicable
        df.loc[((df['Neighbourhood'] == 'Not assigned') & \
                (df['Borough'] == row['Borough'])), 'Neighbourhood'] = row['Borough']
    print("{} neighbourhoods = 'Not assigned' changed to Borough name | no further action needed, correct".format(len(df_dups)))
else:
    print("No entries with 'Not assigned' for Neighbourhood | correct")

No entries with 'Not assigned' for Neighbourhood | correct


### final cell for assignment

In [127]:
# as a final result, print the shape of the dataframe
df.shape

(103, 3)